In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np #linear algebra
import pandas as pd #data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #data visualization
import seaborn as sns  #data visualization
from scipy import stats #stats library
from pylab import rcParams


#Matplotlib runtime(rc) configuration options
rcParams['figure.figsize'] = 11, 9
sns.set_theme(style = "whitegrid")



#Coerce warning issues
import warnings
warnings.filterwarnings('ignore')



# Importing time-based libraries
import time
from datetime import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        

# Libraries for statistical visualization in time-series
from pandas.plotting import autocorrelation_plot as ap, lag_plot
from scipy.stats import boxcox
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.ar_model import AutoReg, AR
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import mean_squared_error


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Suggestions and tips for a forecasting model:

**Summary of 5 step Iterative process and Iterative Forecast Development Process**
* Select or devise a time series forecast process that is tailored to your project, tools, team,
and level of expertise.
*  Write down all assumptions and questions you have during analysis and forecasting work,
then revisit them later and seek to answer them with small experiments on historical data.
* Review a large number of plots of your data at different time scales, zooms, and transforms
of observations in an effort to help make exploitable structures present in the data obvious
to you.
* Develop a robust test harness for evaluating models using a meaningful performance
measure and a reliable test strategy, such as walk-forward validation (rolling forecast).
* Start with simple naive forecast models to provide a baseline of performance for more
sophisticated methods to improve upon.
* Create a large number of perspectives or views on your time series data, including a suite
of automated transforms, and evaluate each with one or a suite of models in order to help
automatically discover non-intuitive representations and model combinations that result
in good predictions for your problem.
* Try a suite of models of differing types on your problem, from simple to more advanced
approaches.
* Try a suite of configurations for a given problem, including configurations that have worked
well on other problems.
* Try automated hyperparameter optimization methods for models to flush out a suite of
well-performing models as well as non-intuitive model configurations that you would not
have tried manually.
* Devise automated tests of performance and skill for ongoing predictions to help to
automatically determine if and when a model has become stale and requires review or
retraining.

# Data Description and Task
Sunspots are temporary phenomena on the Sun's photosphere that appear as spots darker than the surrounding areas. They are regions of reduced surface temperature caused by concentrations of magnetic field flux that inhibit convection. Sunspots usually appear in pairs of opposite magnetic polarity. Their number varies according to the approximately 11-year solar cycle.

Information : https://en.wikipedia.org/wiki/Sunspot
Source: https://www.sidc.be/silso/INFO/snmtotcsv.php

The dataset has been sourced from SIDC website. It seems that the data is captured on a daily basis, and is provided in four formats, as DAILY TOTAL SUNSPOT NUMBER, MONTHLY MEAN SUNSPOT NUMBER, 13-MONTH SMOOTHED MONTHLY TOTAL SUNSPOT NUMBER and YEARLY MEAN TOTAL SUNSPOT NUMBER. All four datasets have different starting periods and the observations have been recorded till the current date. For my analysis, I have chosen the dataset on MONTHLY MEAN SUNSPOT NUMBER 


Data Description:
ADD LATER 
* Task: Predict the requisite number of sunspots for the upcoming month.


TO DO LIST
- [x] Data Cleaning
   - [x] Erroneous data types and value encoding 
- [x] Basic Statistical Analysis
   - [x] Null Values 
   - [x] Duplicates and data types 
   - [x] Description and information 
- [x] Date Time Based Analysis
   - [x] Origin of Timestamps - How are they generated?
   - [x] Time Interval - Regular or Ireegular
   - [x] Extracting Year, Month, Date and Day 
- [x] Preliminary Visualization 
   - [x] Analyze univariate plots(scatter plot, lag plots, histograms, KDE and boxplots) 
   - [x] Heatmpas and Autcorrelation plots 
   - [ ] Analyze the data for different periods - upsampling or downsampling data - DOMAIN SPECIFIC question - ask that to yourself - THIS PART WILL ARRIVE IN DETAILED ANALYSIS section
   - [ ] Check for a distorted distribution after the POWER Transform
   - [ ] Answering why do I want to capture the next month's data rather than the yearly data 
   - [ ] Resampling data and capturing rolling mean statistics 
   - [ ] Predict yearly occurences of average no. of sunspots 
   - [ ] Group all monthly data and capture several statistics - all plots - analyze again 
   - [ ] First group the data and analyze, then perform resampling and derive all statistical inferences 

## LOADING DATA AND DEVELOPMENT OF TEST HARNESS

In [ ]:
__author__ = "Sagar Sinha"
__NOTEBOOK__ = "Univariate Time Series Prediction"

In [ ]:
data = pd.read_csv("../input/sunspots/Sunspots.csv", parse_dates=True, infer_datetime_format=True, skip_blank_lines=True) # Reading the dataset
curr_data = pd.read_csv("../input/sunspot-no-till-current-date/SN_m_tot_V2.0.csv", parse_dates=True, infer_datetime_format=True, skip_blank_lines=True)
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.columns = ['date', 'no_of_sunspots']
data.drop(3264, axis=0, inplace=True) # Dropping the last row as there exists only a single value with that year value. It would introduce certain issues later on 

In [ ]:
curr_data.columns = curr_data.columns.str.split(";")
curr_data.columns = ["Combined_Data"]
curr_data[["Year", "Month", "Frac", "Sunspots", "NR_1", "NR_2", "NR_3"]] = curr_data["Combined_Data"].str.split(";", expand = True)
curr_data.drop(["Combined_Data", "Frac", "NR_1", "NR_2", "NR_3"], axis = 1, inplace = True)

In [ ]:
### When we would require it, format the datetime column 
### We only need the current data from February 2021 onwards. This is the dataset for prediction 
curr_data_req = curr_data[3263: ]
### Rename column of current data
curr_data_req = curr_data_req.rename(columns = {"Sunspots": "no_of_sunspots"})

In [ ]:
print(data.isnull().sum())  # Check for null values
print("\n")
print(curr_data_req.isnull().sum())
print("\n")
print(data.dtypes)
print("\n")
print(curr_data_req.dtypes)

**The 'Date' column is of 'object' data type. It needs to be converted into the datetime format**

In [ ]:
data['date'] = data['date'].astype('datetime64') # Type-casting ['date'] column to Pandas datetime format
data = data.drop_duplicates() # Removes all duplicate rows, if any 

In [ ]:
## Convert data into a series
data.set_index(['date'], inplace=True) # Setting 'date' as the index column 
data.index.name = None
sunspots = data['no_of_sunspots']

In [ ]:
print(data.describe())
print()
print(data.info())

In [ ]:
### We have the training and test datasets. Let's create the validation data set - We would split the dataset as: TRAINING=0.7%, VALIDATION=0.2%, TEST=0.1%
#### No. of training samples = 3234
#### No. of validation samples = 34
#### No. of test samples = 17
train_data = data[0:3240]
val_data = data[3240: ]
test_data = curr_data_req

In [ ]:
### Choice of evaluation metric = RMSE. Since we want to count of sunspots
### Development of persistence model on validation data
val_data["no_of_sunspots_lagged"] = val_data["no_of_sunspots"].shift(1)

### Backfilling row data consisting of NaN values
val_data_persist = val_data.bfill(axis = "rows")

#### Store predictions
truth_vals = np.array([value for value in val_data_persist["no_of_sunspots"]])
pred_vals = np.array([value for value in val_data_persist["no_of_sunspots_lagged"]])

#### Calculate RMSE and plot the results 
rmse = np.sqrt(mean_squared_error(truth_vals, pred_vals))

#### Plotting the results   
fig, (ax1, ax2) = plt.subplots(2, 1)
line1, = ax1.plot(truth_vals[0:500], color = "r", linestyle = "-", alpha = 0.8)
line2, = ax2.plot(pred_vals[0:500], color = "y", linestyle= "-", alpha = 0.8)
plt.show()

In [ ]:
print(rmse)

In [ ]:
### Create a lagged variable for the validation dataset 
val_data["no_of_sunspots_lagged"] = val_data["no_of_sunspots"].shift(1)
val_data = val_data.bfill(axis = "rows")

### Observations
The baseline model is pretty close to the original one, and it serves us a good starting point 
* The baseline RMSE value is 6.576

## PRELIMINARY VISUALIZATIONS AND OBSERVATIONS

In [ ]:
## Determine the time interval and frequency of occurence of sunspots 
## The underlying assumption is that the data has a monthly frequency 
## Plot the entire series
## Group the data at different scales and then plot the series
train_data.plot()

In [ ]:
### Reset the indexes 
# train_data = train_data.set_index("date")
# val_data = val_data.set_index("date")

In [ ]:
## Let's plot yearly data, i.e., say for 1749 
year_data = train_data["1749-01-31": "1749-12-31"]
year_data.plot()

In [ ]:
train_data["1749-1-31": "1760-1-31"].min()

In [ ]:
## According to an observation regarding sunspots, they follow an 11 month cycle, which is called the solar cycle 
## Caqpture 11 years from 1769 to 1780 
fig, ax = plt.subplots(1, figsize=(20, 10))
year_range_sunspots_1 = train_data["1749-1-31": "1760-1-31"]
ax.plot(year_range_sunspots_1, color="red")
ax.axhline(y = year_range_sunspots_1.values.min(), linestyle = "-", color="g")
ax.axhline(y = year_range_sunspots_1.values.max(), linestyle = "-", color="y")
ax.text(x = year_range_sunspots_1.idxmax(), y = year_range_sunspots_1.values.max(), s="MAX", horizontalalignment="left", verticalalignment="top", fontsize=20, color="blue")
ax.text(x = year_range_sunspots_1.idxmin(), y = year_range_sunspots_1.values.min(), s="MIN", horizontalalignment="right", verticalalignment="bottom", fontsize=20, color="blue")

In [ ]:
## According to an observation regarding sunspots, they follow an 11 year cycle, which is called the solar cycle 
## Capture 11 years from 1769 to 1780 
plt.figure(figsize=(20, 10))
year_range_sunspots_1 = train_data["1749-1-31": "1760-1-31"]
year_range_sunspots_2 = train_data["1760-2-1": "1772-1-31"]
year_range_sunspots_1.plot()
year_range_sunspots_2.plot()
plt.xlabel("Variation of average no. of sunspots over 11 year period")
plt.ylabel("No. of sunspots")

### Observations
* It is interesting to note that the series has a stepwise fall in frequency of sunspots - prevalece of a large no. of sunspots has a lower frequency
* Hence, there occurs only a few instances when the no. of sunspots are large in number 
* The series has an underlying cyclic feature. It is an indicative of the probable prevalence of seasonality component within data 
* Furthermore, we can observe that there is a maxima and a minima in an yearly period as well as in an 11 year period - the latter is known as solar cycle
* Every 11 years, the no. of sunspots reach a maximum limit suddenly, and then gradually decreases to a minimum limit, rises suddenly and the cycle follows

In [ ]:
# Group monthly data into yearly period
## Convert sunspots to DataFrame 
## Drop rows for the year 2018 from training data 
annual_groups = train_data["no_of_sunspots"].groupby(pd.Grouper(freq='A'))
year_group_data = {}

for index, group in annual_groups:
    year_group_data[index.year] = group.values
    
year_group_df = pd.DataFrame(year_group_data)
year_group_df = year_group_df.T

In [ ]:
year_group_df["avg_over_year"] = year_group_df.sum(axis = 1)/12
year_group_df

In [ ]:
## Average number of sunspot values over the entire period
avg_by_year = {}
for year in year_group_df.columns:
    avg_by_year[year] = year_group_df[year].mean()

plt.plot(avg_year.values)

# Plotting average by mean gives us a smoother result which can help us in predicting future yearly average values of solar spots 

In [ ]:
avg_by_year.values()

In [ ]:
## Plotting autocorrelation plot
ap(avg_year)

In [ ]:
ap(train_data)

In [ ]:
plt.subplot(121)
lag_plot(avg_year)

### Print lag plot of the original dataset 
plt.subplot(122)
lag_plot(train_data)

In [ ]:
### set the spacing between subplots
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)

### The statistical relationship between avg spot no. of a particular month to a preceding month seems highly significant. Let's plot for multiple lags, say for upto 9 
i, j, k = 3, 3, 1

for i in range(0, 9):
    plt.subplot(330 + k)
    lag_plot(train_data.shift(k))
    plt.xlabel("t")
    plt.ylabel("y(t-"+str(k)+")")
    k += 1       

In [ ]:
### Analyze a boxplot for the same 
plt.boxplot(train_data)

In [ ]:
### KDE plot of original data
train_data.plot(kind = "kde")

In [ ]:
### Histogram plot of original data
train_data.plot(kind = "hist")

In [ ]:
train_data = pd.DataFrame(train_data)
train_data.columns = ["Sunspots"]

### Observations
* We have capped and removed the outliers, but the distribution has been distorted. Use Power Transforms later

## Time Series decomposition 

In [ ]:
### Decompose the dataframe into its required constituents 
sunspots_decomposed = seasonal_decompose(train_data["no_of_sunspots"], model = "additive", freq=30 )

In [ ]:
### Plotting decomposition 
sunspots_decomposed.plot()

### Observations
* We can notice that there exists a periodic change in correlation from positive to negative in avg. no. of sunspots, with highly appreciable correlation values, for around 500 days
* It coudl be that we need to downsample monthly data to an yearly one for incorportating this observation to the fullest. But for understanding purposes, let's model both, i.e., for yearly as well as diurnal data 
* The dataset has a strong trend as well as seasonal component 
* There are some occasional aberrations in the trend component between 1800-1840 and at some other places as well
* There is a uniform seasonal component that can be removed. Removing the seasonal component may aid in smoothening the data  
* The residual values seem to be uniformly distributed with constant variance, which is an indication that it doesn't have any time series component and doesn't contribute in forecasting. Hence, it can be directly removed. 

In [ ]:
### Exploring the residual data 
train_data["resid"] = sunspots_decomposed.resid

### Impute NaN values in residuals with 0, as 0 is indicative of the absence of residual values 
train_data["resid"] = train_data["resid"].fillna(0)

### Exploring seasonal data
train_data["seasonal"] = sunspots_decomposed.seasonal

###  Impute NaN values in seasonal with 0, as 0 is indicative of the absence of seasonality 
train_data["seasonal"] = train_data["seasonal"].fillna(0)

In [ ]:
### Removing resduals 
train_data["de_resid"] = train_data["no_of_sunspots"] - train_data["resid"]

### Removing seasonality 
train_data["de_seasonal"] = train_data["de_resid"] - train_data["seasonal"]

In [ ]:
### Plotting the transformed data 
train_data["de_seasonal"].plot(kind = "kde")

In [ ]:
### Cap values beyond some positive and negative values and re-draw the curve 
upper_limit = 1.5 * np.mean(train_data["Sunspots"])
train_data["Sunspots"] = np.where(train_data["Sunspots"] > upper_limit, upper_limit, train_data["Sunspots"])
# train_data["Sunspots"] = np.where(train_data["Sunspots"] < 200, 200, train_data["Sunspots"])

sunspots.plot(kind="box")
# sunspots.plot(kind="kde")

## Checking Stationarity of Time Series

In [ ]:
test_results = adfuller(train_data["de_seasonal"])
test_results

In [ ]:
### Printing the test statistics
print("The critical value is %0.5f" % (test_results[0]))
print("The p-value is %0.5f" % (test_results[1]))

### Observation: 
* As we can observe, the test-statistic is quite less than Z-value at p=0.01 or p=0.05. Hence we fail to accept the null hypothesis and declare that the series is stationary 
* No transformation or differencing is required 

## Time Series Modelling 

In [ ]:
### Checking for AR and MA 
#### Since the model had no residuals, as observed during seasonal decomposition, it is clearly not an MA model 
#### Let's check for AR instead
plot_pacf(train_data["de_seasonal"])

In [ ]:
plot_acf(train_data["de_seasonal"])

### Observations:
* As observed, the PACF plot shows a sharp decline in correlation, with an acceptable lag value of 25-28, whereas the ACF correlation values decline gradually over a longer period of time. Hence we will forecast the data on AR model with a lag value(p) of around 25-28 for our model 

In [ ]:
### AR model
train_values = train_data["de_seasonal"]
val_values = val_data["no_of_sunspots"]
history = [x for x in train_values]

In [ ]:
model = AutoReg(history, lags = 28).fit()    
preds = model.predict(start = len(train_values) + 1, end = len(train_values) + len(val_values), dynamic = False)

### Some prediction values are negative. But negative values are unacceptable as sunspot numbers can't be negative 
preds_final = []
for pred in preds:
    if pred < 0:
        preds_final.append(0)
    else:
        preds_final.append(pred)
print(np.sqrt(mean_squared_error(val_data["no_of_sunspots"].values, preds_final)))

### Observations:
* The RMSE for validation data with AR model is comaprable with the baseline model. We can still improve the model via - walk forward cross validation 

In [ ]:
for pred in preds:
    history.append(pred)
test_data["no_of_sunspots"] = test_data["no_of_sunspots"].astype(float)

In [ ]:
model = AutoReg(history, lags = 128).fit()
preds_test = model.predict(start = len(train_values) + len(val_values) + 1, end = len(train_values) + len(val_values) + len(test_data), dynamic = False)
pred_test_final = []
for pred in preds_test:
    
print(np.sqrt(mean_squared_error(test_data["no_of_sunspots"].values, preds_test)))

In [ ]:
plt.plot(test_data["no_of_sunspots"].values)
plt.plot(preds_test)

# Function to add a legend  
plt.legend(["Original Test Observations", "Test Predictions"], loc ="lower right")

### Plotting Residuals

In [ ]:
test_data["no_of_sunspots"].values[0]

In [ ]:
test_data["no_of_sunspots"][0]

In [ ]:
test_resids = [test_data["no_of_sunspots"].values[i] - preds_test[i] for i in range(len(test_data))]

### Plotting the kdeplot 
sns.kdeplot(test_resids)

In [ ]:
### Plotting histogram 
sns.distplot(test_resids)

### Observations:
* The test set residuals follow an almost normal distibution
* Still, the trend component hasn't been perfectly captured and the model  

In [ ]:
### Check for presence of lags in autocorrelation and partial autocorrelation plots 
plot_acf(val_data["resid"], lags = 10)

In [ ]:
plot_pacf(val_data["resid"], lags = 10)

## Power Transformation 

In [ ]:
### Add the code for boxcox transformation 

### Observations
1. Box-Cox transformation = yields values in a lower range but it doesn't smoothen out the data much 
2. The only thing that works in our favour is that the range of values has reduced that would aid in the efficient learning of a model 

In [ ]:
### Using box-cox transformations to identify the ideal value for lambda. A simple transformation won't work here. Let the model find the ideal transformation for us 
#### It is worthy to note that the transformation only considerspositive values. Sincde tehre are several zeroes as values, we need to add 1 to the entire dataset and then pass it to the boxcox function
train_data["sunspots_transform"], lamb = boxcox(train_data["no_of_sunspots"]+1)
train_data["sunspots_transform"] = boxcox(train_data["no_of_sunspots"]+1, lmbda = lamb)

In [ ]:
### Separate out the independent and dependent variables. Let the lagged values be the independent variable and the actual no. of sunspots as the dependent variable 
# for i in range(len(val_values)):
#     ar_model = AutoReg(history, lags = 6).fit()
#     yhat = ar_model.predict(start = len(train_values) + i + 1, end = len(train_values) + len(val_values) + i + 1, dynamic = False)
#     preds.append(yhat)
#     history.append(yhat)
# X_train, y_train = train_data_persist["no_of_sunspots_lagged"].values, train_data_persist["no_of_sunspots"].values
# history = [x for x in train_data_persist["no_of"]]
# X_val, y_val = val_data_persist.iloc[:, 1].values, val_data_persist.iloc[:, 0]
# ar_model = AutoReg(y_train, lags=6, trend="ct")
# ar_model.fit(X_train, y_train)
# preds = ar_model.predict(val_data_persist["no_of_sunspots"])

**Reading in the data and basic formatting**

In [ ]:
sunspots = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv', sep=',', parse_dates=True)

In [ ]:
sunspots.head()

### Observations

In [ ]:
sunspots['Month'] = pd.to_datetime(sunspots['Month'])
sunspots.set_index('Month')

In [ ]:
sunspots.tail()

# Visualization of a time-series, and summary stats and diagnostics

In [ ]:
med_value = sunspots['Sunspots'].median()
quant_25 = sunspots['Sunspots'].quantile(0.25)
quant_99 = sunspots['Sunspots'].quantile(0.99)

In [ ]:
#Setting up the style of background grid
fig = plt.figure()
plt.style.use('fivethirtyeight')
ax = sunspots['Sunspots'].plot(color='blue', fontsize=10, figsize=(10, 8))


#A horizontal span 
#Also there are 
ax.axvspan(quant_25, quant_99, color='green', alpha=0.3)

#A vertical span
ax.axhspan(30, 150, color='red', alpha=0.3)

In [ ]:
#Checking if there are any null values
sunspots.isnull().sum()

There aren't any null values in the dataframe

**Window Functions:**
1. Used to identify sub-periods, calculates sub-metrics of sub-periods.
2. Rolling - same size and sliding
3. Expanding - includes all previous values

In [ ]:
#Setting the index back to datetime format
sunspots = sunspots.set_index('Month')

In [ ]:
#Rolling mean visualizations of some section of data, say from 1749 to 1753, unable to understand the concept
sunSome_part = sunspots[1749 : 1753]


sunSome_mean1 = sunSome_part.rolling(1).mean() #rolling mean for a single month
sunSome_mean2 = sunSome_part.rolling(2).mean() #rolling mean for 2 consecutive months
sunSome_mean3 = sunSome_part.rolling(3).mean() #rolling mean for a period of 3 months

# ax = sunSome_mean1.plot()
# ax.set_xlabel('Date')
# ax.set_ylabel('Rolling Mean Variation')
# ax.set_title('SPOT statistics')

plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(1, figsize=(10, 5))

ax.plot(sunSome_mean1, sunSome_mean1.index, linewidth=2, markersize=12, color='green')
ax.plot(sunSome_mean2, sunSome_mean2.index, linewidth=2, markersize=12, color='blue')
ax.plot(sunSome_mean3, sunSome_mean3.index, linewidth=2, markersize=12, color='red')

plt.title('Plotting out the rolling mean statistics')
plt.xlabel('Susnpots Mean')
plt.ylabel('Period')
plt.legend(['1Day', '2Day', '3Day'])

plt.show()

In [ ]:
#Let's plot a more compact representation of our data. Here we will be computing rolling avergae for a lagging period of 2 months, i.e, 60 days.
#ma variable is for moving avergaes
ma = sunspots.rolling(window=2).mean()
mstd = sunspots.rolling(window=2).std()

#Adding the lower bound
ma['Lower'] = ma['Sunspots'] - (2 * mstd['Sunspots'])

#Adding the upper bound
ma['Upper'] = ma['Sunspots'] + (2 * mstd['Sunspots'])

#Plot the dataframe and set the labels
plt.figure(figsize=(10, 5))

ax = ma.plot(linewidth=0.8, fontsize=6)
plt.xlabel('Date')
plt.ylabel('Number of sunspots')
plt.xlabel('Date')
plt.title('Rolling mean and variance of the number of sunspots over the given period of time')

plt.show()

**Plotting aggregate values of a time series**

In [ ]:
#For our use-case, let's try to plot the aggregate values for the number of spots in the year 1750
sunspots_1750 = sunspots.iloc[sunspots.index.year==1750, : ]
index_month = sunspots.index.month
sunspots_1750_by_month = sunspots.groupby(index_month).Sunspots.mean()

sunspots_1750_by_month.plot()
plt.ylabel('Cases per month')
plt.legend('Sunspots', loc='upper left')
plt.show()

Hence from the above visualization we can infer that the number of sunspots is at peak during the summer months,  which could be predetermindedly hypothesised.

Summarizing and plotting summary statistics

In [ ]:
#Describing the dataframe
print(sunspots.describe())

#Minmimum value
print(sunspots['Sunspots'].min())

#Maximum value
print(sunspots['Sunspots'].max())

In [ ]:
#Printing out the boxplot for visualizing summary statistics
boxplot = sunspots.boxplot()

In [ ]:
sunspots_copy = sunspots.copy()
sunspots_copy.head()

In [ ]:
sunspots_copy.rename(columns={'Month':'Date'}, inplace=True)

In [ ]:
#Printing out the boxplot for visualizing summary statistics
boxplot = sunspots_copy.boxplot()

In [ ]:
#We observe there are outliers in the upper half of boxplot. Let's reassign the outlier values to be equal to the upper half of the boxplot.
upper_perc = sunspots_copy['Sunspots'].quantile(0.75)
lower_perc = sunspots_copy['Sunspots'].quantile(0.25)

upper_limit = upper_perc + (3 * upper_perc)
lower_limit = lower_perc - (3 * lower_perc)

print(upper_limit)
print(lower_limit)

In [ ]:
#If the value of skewness is above 1, then it means there exists a positive skewness.
sunspots_copy['Sunspots'].skew()

Histograms and Kernel Density Estimations(KDE):

In [ ]:
sunspots['Sunspots'].plot(kind='hist', bins=100)
plt.show()

In practice, histograms can be a substandard method for assessing the distribution of your data because they can be strongly affected by the number of bins that have been specified. Instead, kernel density plots represent a more effective way to view the distribution of your data. An example of how to generate a density plot of is shown below:

In [ ]:
ax = sunspots['Sunspots'].plot(kind='density', linewidth=2)

In [ ]:
#Since the distribution isn't normal, we will perform the following the quantile-based imputation
#sunspots_copy[sunspots_copy['Sunspots'] > upper_limit] = upper_limit

In [ ]:
#There are a lot of negative values in the dataset. We need to remove them for the transformation to happen
#sunspots_copy['Sunspots-1'] = sunspots_copy[sunspots_copy['Sunspots'] > 0]

In [ ]:
#Log-trasnform for removing left skewness
#sunspots_copy['Sunspots-1'] = np.log1p(sunspots_copy['Sunspots-1'])

In [ ]:
#sunspots_copy['Sunspots-1'].plot(kind='hist', bins=100)
#plt.show()

In [ ]:
#sunspots_copy.drop(['Sunspots'], axis=1, inplace=True)

In [ ]:
#sunspots_copy['Sunspots-1'].plot(kind='kde')

Plotting autocorrelation and autocorrelation

In [ ]:
#Plotting autocorrelation
#Lags and alpha are the only important parameters in these plots
fig = plot_acf(sunspots_copy['Sunspots'], lags=20)
plt.show()

In [ ]:
#Plotting partial autocorrelation
#Lags and alpha are the only important parameters in these plots
fig = plot_pacf(sunspots_copy['Sunspots'], lags=20)

Like autocorrelation, the partial autocorrelation function also measures the correlation coefficient between a time series and a lagged version of itself. But
the main difference between the two is that PACF smoothens(lessens variations) the effect of lags beyond the ones explicitly mentioned.

**Time Series Decomposition** - for visualizing trend, seasonality and noise

In [ ]:
rcParams['figure.figsize'] = 11, 9

decomposition = seasonal_decompose(sunspots['Sunspots'])
figure = decomposition.plot()
plt.show()

In [ ]:
print(dir(decomposition))

In [ ]:
print(decomposition.seasonal)

Time Series decomposition is a powerful tool to reveal the structure in a time-series.

In [ ]:
#A seasonal component(cyclic component) exists when a time-series is influenced by seasonal factors. 
decomp_seasonal = decomposition.resid
ax = decomp_seasonal.plot(figsize=(14, 10))
ax.set_xlabel('Date')
ax.set_ylabel('Seasonality')
ax.set_title('Seasonal values of the time series')
plt.show()

So far we have known:
1. Visualize aggregates of time series data
2. Extract statistical summaries
3. Autocorrelation and Partial autocorrelation
4. Time Series decomposition.

Multiple time-****series plots - refer the course

**Also you can print out the relationships between different time series data using heatmaps and clustered heatmaps.**
1. Create facetted plots and graphs(using the pandas .plot function and setting up the layout of plots
2. Set horiziontal/vertical lines/regions to specify/highlight some important year/date. This is ideal for a multiple time-series dataset.
3. Aggregate plots are also ideal for a time-series dataset. (Monthly or yearly trends) alongwith bbox_to_anchor)
4. Seasonal decomposition of multiple time-series togther.

Multiple time-series visualizations and code templates

> Time-Series Visualizations

In [ ]:
# Plot all time series in the jobs DataFrame
# ax = jobs.plot(colormap='Spectral', fontsize=6, linewidth=0.8)
    
# Set labels and legend
# ax.set_xlabel('Date', fontsize=10)
# ax.set_ylabel('Unemployment Rate', fontsize=10)
# ax.set_title('Unemployment rate of U.S. workers by industry', fontsize=10)
# ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
# Annotate your plots with vertical lines
# ax.axvline('2001-07-01', color='blue', linestyle='--', linewidth=0.8)
# ax.axvline('2008-09-01', color='blue', linestyle='--', linewidth=0.8)

# Show plot
# plt.show()

In [ ]:
# Extract the seasonal values for the decomposition of each time series
# for ts in jobs_names:
#     jobs_seasonal[ts] = jobs_decomp[ts].seasonal
    
# Create a DataFrame from the jobs_seasonal dictionary
# seasonality_df = pd.DataFrame.from_dict(jobs_seasonal)

# Remove the label for the index
# seasonality_df.index.name = None

# Create a faceted plot of the seasonality_df DataFrame
# seasonality_df.plot(subplots=True,
#                    layout=(4, 4),
#                    sharey=False,
#                    fontsize=2,
#                    linewidth=0.3,
#                    legend=False)

# Show plot
# plt.show()

In [ ]:
# Get correlation matrix of the seasonality_df DataFrame
# seasonality_corr = seasonality_df.corr(method='spearman')

# Customize the clustermap of the seasonality_corr correlation matrix
# fig = sns.clustermap(seasonality_corr, annot=True, annot_kws={"size": 4}, linewidths=.4, figsize=(15, 10))
# plt.setp(fig.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
# plt.setp(fig.ax_heatmap.xaxis.get_majorticklabels(), rotation=90)
# plt.show()

# Print the correlation between the seasonalities of the Government and Education & Health industries
# print(0.89)

In [ ]:
#Unlabelling the indices
#The packages to be used are pandas, numpy, statsmodels and scipy for linear regression.

In [ ]:
#Probable questions to be asked
#1. Do I need to resaqmple in my use-case?
#2. Do I need to apply percent changes in my use-case?
#3. 